In [37]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [38]:
def multiprocessing_run(prior, M):
    
    import time
    import os
    from multiprocessing import Pool, Array

    # They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
    import warnings
    warnings.filterwarnings("ignore")

    np.random.seed(12345)
    n_iterations = 16

    iterations_set = np.arange(1,n_iterations+1,1)
    M = (M,)
    prior_list = [prior]#, "lasso_alternative", "horseshoe"]
    args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

    if __name__ == '__main__':

        pool = Pool()
        simulation_results = pool.starmap(simulation_run, args_tuples)

#### M = 3, $p_{0} = 0.40$ and $T = 200$

In [3]:
M = 3

In [4]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [5]:
%%time
multiprocessing_run("svss",M)

Run: 9, M: 3 & prior: svss -> MSD: 0.03352099252120182 | MSFE: 3.7437281100614888 | ALPL: -36.13508504511776 | elapsed time: 301.36288
Run: 2, M: 3 & prior: svss -> MSD: 0.029495293907834404 | MSFE: 1.41356664660755 | ALPL: -34.56565377213802 | elapsed time: 342.387834
Run: 4, M: 3 & prior: svss -> MSD: 0.05597547196652418 | MSFE: 2.3277093668888673 | ALPL: -35.307029592121836 | elapsed time: 355.276626
Run: 13, M: 3 & prior: svss -> MSD: 0.03358257793545593 | MSFE: 1.7268947336139409 | ALPL: -36.43877883578196 | elapsed time: 357.924236
Run: 16, M: 3 & prior: svss -> MSD: 0.041516006371082464 | MSFE: 2.383456488152457 | ALPL: -33.45613949485876 | elapsed time: 420.449813
Run: 11, M: 3 & prior: svss -> MSD: 0.05536645606591423 | MSFE: 1.2691867936921764 | ALPL: -35.09680877903058 | elapsed time: 513.99691
Run: 3, M: 3 & prior: svss -> MSD: 0.08969874374371223 | MSFE: 2.55126047156182 | ALPL: -36.40781163350265 | elapsed time: 515.214553
Run: 8, M: 3 & prior: svss -> MSD: 0.052387322413

In [6]:
%%time
multiprocessing_run("lasso_alternative",M)

Run: 1, M: 3 & prior: lasso_alternative -> MSD: 0.027986271717716203 | MSFE: 1.9880027118174028 | ALPL: -23.340640953148764 | elapsed time: 65.733701
Run: 9, M: 3 & prior: lasso_alternative -> MSD: 0.02820214745258613 | MSFE: 3.71490417767514 | ALPL: -28.53623354022198 | elapsed time: 69.442243
Run: 4, M: 3 & prior: lasso_alternative -> MSD: 0.0530779891215395 | MSFE: 2.3143464931649707 | ALPL: -27.536175175636544 | elapsed time: 70.030683
Run: 13, M: 3 & prior: lasso_alternative -> MSD: 0.046518734272515634 | MSFE: 1.7760972667450983 | ALPL: -30.11064257639115 | elapsed time: 71.46176
Run: 2, M: 3 & prior: lasso_alternative -> MSD: 0.03088248494795688 | MSFE: 1.4045091395489626 | ALPL: -26.527053129654597 | elapsed time: 73.458084
Run: 15, M: 3 & prior: lasso_alternative -> MSD: 0.04856946585251897 | MSFE: 2.0160612251457763 | ALPL: -23.07549745325975 | elapsed time: 74.710826
Run: 7, M: 3 & prior: lasso_alternative -> MSD: 0.04051883657190872 | MSFE: 1.8414694134827967 | ALPL: -16.73

In [7]:
%%time
multiprocessing_run("horseshoe",M)

Run: 2, M: 3 & prior: horseshoe -> MSD: 0.033201053761108476 | MSFE: 1.3863125095973348 | ALPL: -27.74216677195501 | elapsed time: 828.858483
Run: 1, M: 3 & prior: horseshoe -> MSD: 0.03175627859111631 | MSFE: 1.9556107325317016 | ALPL: -26.998382681226055 | elapsed time: 829.953958
Run: 3, M: 3 & prior: horseshoe -> MSD: 0.03578424842552312 | MSFE: 2.5231187694951034 | ALPL: -27.76023746304406 | elapsed time: 830.388327
Run: 4, M: 3 & prior: horseshoe -> MSD: 0.05393796238180104 | MSFE: 2.2848708150683787 | ALPL: -29.334867097583512 | elapsed time: 833.421688
Run: 5, M: 3 & prior: horseshoe -> MSD: 0.04533960260468031 | MSFE: 1.8044584241664907 | ALPL: -31.51372830761033 | elapsed time: 833.699402
Run: 13, M: 3 & prior: horseshoe -> MSD: 0.046505574908750374 | MSFE: 1.7653079333846935 | ALPL: -30.389196165895353 | elapsed time: 834.251563
Run: 15, M: 3 & prior: horseshoe -> MSD: 0.04389736311390241 | MSFE: 1.9797008802138623 | ALPL: -26.95664327252537 | elapsed time: 835.186646
Run: 1

#### M = 3, $p_{0} = 0.20$ and $T = 200$

In [8]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [9]:
multiprocessing_run("svss",M)

Run: 9, M: 3 & prior: svss -> MSD: 0.036933573948727406 | MSFE: 3.8021424476127104 | ALPL: -36.18633008950053 | elapsed time: 290.648822
Run: 2, M: 3 & prior: svss -> MSD: 0.029495293907834404 | MSFE: 1.41356664660755 | ALPL: -34.56565377213802 | elapsed time: 342.510498
Run: 13, M: 3 & prior: svss -> MSD: 0.03358257793545593 | MSFE: 1.7268947336139409 | ALPL: -36.43877883578196 | elapsed time: 359.294462
Run: 4, M: 3 & prior: svss -> MSD: 0.061530803407856695 | MSFE: 2.8510873882395655 | ALPL: -34.412524611522564 | elapsed time: 394.242802
Run: 3, M: 3 & prior: svss -> MSD: 0.07806857803930205 | MSFE: 3.4411857470018203 | ALPL: -35.31775787422632 | elapsed time: 405.59164
Run: 8, M: 3 & prior: svss -> MSD: 0.03671308830395939 | MSFE: 1.181050347705105 | ALPL: -36.066231399568416 | elapsed time: 413.070297
Run: 16, M: 3 & prior: svss -> MSD: 0.041516006371082464 | MSFE: 2.383456488152457 | ALPL: -33.45613949485876 | elapsed time: 419.318406
Run: 1, M: 3 & prior: svss -> MSD: 0.06284723

In [10]:
multiprocessing_run("lasso_alternative",M)

Run: 9, M: 3 & prior: lasso_alternative -> MSD: 0.027996929148910554 | MSFE: 3.7476419718316594 | ALPL: -28.897062561866292 | elapsed time: 67.291793
Run: 1, M: 3 & prior: lasso_alternative -> MSD: 0.026696801215703562 | MSFE: 2.0789016084853236 | ALPL: -24.24113209941011 | elapsed time: 69.297642
Run: 13, M: 3 & prior: lasso_alternative -> MSD: 0.046518734272515634 | MSFE: 1.7760972667450983 | ALPL: -30.11064257639115 | elapsed time: 71.68597
Run: 4, M: 3 & prior: lasso_alternative -> MSD: 0.05096729847487726 | MSFE: 2.751012959238011 | ALPL: -26.751162223881877 | elapsed time: 72.664982
Run: 2, M: 3 & prior: lasso_alternative -> MSD: 0.03088248494795688 | MSFE: 1.4045091395489626 | ALPL: -26.527053129654597 | elapsed time: 73.989742
Run: 15, M: 3 & prior: lasso_alternative -> MSD: 0.04856946585251897 | MSFE: 2.0160612251457763 | ALPL: -23.07549745325975 | elapsed time: 75.567361
Run: 16, M: 3 & prior: lasso_alternative -> MSD: 0.038881679677575255 | MSFE: 2.220914469785766 | ALPL: -2

In [11]:
multiprocessing_run("horseshoe",M)

Run: 4, M: 3 & prior: horseshoe -> MSD: 0.04719875406043033 | MSFE: 2.7230648076434374 | ALPL: -28.8252577061845 | elapsed time: 837.883113
Run: 10, M: 3 & prior: horseshoe -> MSD: 0.051047256030556265 | MSFE: 1.9833372617389338 | ALPL: -28.24181587368845 | elapsed time: 838.840291
Run: 6, M: 3 & prior: horseshoe -> MSD: 0.054590072963382814 | MSFE: 2.456760058055936 | ALPL: -21.491278330012456 | elapsed time: 839.110152
Run: 3, M: 3 & prior: horseshoe -> MSD: 0.03402956103922543 | MSFE: 3.3638201437179376 | ALPL: -28.326079429063384 | elapsed time: 839.144788
Run: 7, M: 3 & prior: horseshoe -> MSD: 0.056345473550832796 | MSFE: 1.7107319907868546 | ALPL: -20.14890092846447 | elapsed time: 839.418515
Run: 8, M: 3 & prior: horseshoe -> MSD: 0.04614824330003289 | MSFE: 1.2040509732821532 | ALPL: -29.22113372930519 | elapsed time: 839.895527
Run: 12, M: 3 & prior: horseshoe -> MSD: 0.03371322037820577 | MSFE: 0.7685414825182932 | ALPL: -31.461605399584315 | elapsed time: 840.082224
Run: 11

#### M = 3, $p_{0} = 0.40$ and $T = 100$

In [12]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [13]:
multiprocessing_run("svss",M)

Run: 5, M: 3 & prior: svss -> MSD: 0.052717697786817355 | MSFE: 2.656869478774109 | ALPL: -34.823538211402266 | elapsed time: 124.002173
Run: 4, M: 3 & prior: svss -> MSD: 0.03945326401986126 | MSFE: 0.7472732823881812 | ALPL: -35.56857958874501 | elapsed time: 145.07048
Run: 3, M: 3 & prior: svss -> MSD: 0.042213670457323035 | MSFE: 1.8135400408528584 | ALPL: -35.531687085729644 | elapsed time: 150.747431
Run: 12, M: 3 & prior: svss -> MSD: 0.036266041990926784 | MSFE: 1.423609765728122 | ALPL: -35.63634390236838 | elapsed time: 155.196322
Run: 9, M: 3 & prior: svss -> MSD: 0.039115897571197086 | MSFE: 1.2653566673561203 | ALPL: -35.64400447133617 | elapsed time: 163.917435
Run: 7, M: 3 & prior: svss -> MSD: 0.047984211337493934 | MSFE: 1.3782809073434896 | ALPL: -34.76971409300489 | elapsed time: 177.823391
Run: 14, M: 3 & prior: svss -> MSD: 0.04855269229019527 | MSFE: 1.2998028422759509 | ALPL: -34.69807853585111 | elapsed time: 178.867116
Run: 8, M: 3 & prior: svss -> MSD: 0.03556

In [14]:
multiprocessing_run("lasso_alternative",M)

Run: 4, M: 3 & prior: lasso_alternative -> MSD: 0.03373209926996448 | MSFE: 0.7556040658806228 | ALPL: -27.48468049287802 | elapsed time: 31.509284
Run: 2, M: 3 & prior: lasso_alternative -> MSD: 0.05034403187770087 | MSFE: 2.0384332919866495 | ALPL: -19.502053667194474 | elapsed time: 31.793823
Run: 3, M: 3 & prior: lasso_alternative -> MSD: 0.04235384309642415 | MSFE: 1.7572164875771947 | ALPL: -29.41930999831404 | elapsed time: 31.842527
Run: 1, M: 3 & prior: lasso_alternative -> MSD: 0.03043428679556112 | MSFE: 1.4341348809351508 | ALPL: -30.155914102454737 | elapsed time: 32.0917
Run: 11, M: 3 & prior: lasso_alternative -> MSD: 0.029585216333440516 | MSFE: 1.8304520067829921 | ALPL: -27.037458021180868 | elapsed time: 32.662606
Run: 8, M: 3 & prior: lasso_alternative -> MSD: 0.040653464407163706 | MSFE: 3.2251080775428758 | ALPL: -26.777420982584722 | elapsed time: 32.800198
Run: 5, M: 3 & prior: lasso_alternative -> MSD: 0.03376925031790566 | MSFE: 2.780475038405445 | ALPL: -23.7

In [15]:
multiprocessing_run("horseshoe",M)

Run: 14, M: 3 & prior: horseshoe -> MSD: 0.04352989950743131 | MSFE: 1.2845069393355946 | ALPL: -28.79474065830169 | elapsed time: 387.610666
Run: 15, M: 3 & prior: horseshoe -> MSD: 0.03689005432869582 | MSFE: 1.885855575774963 | ALPL: -31.13777186862921 | elapsed time: 387.897618
Run: 9, M: 3 & prior: horseshoe -> MSD: 0.0348653900194121 | MSFE: 1.2624022032147066 | ALPL: -28.19326408226867 | elapsed time: 388.427358
Run: 7, M: 3 & prior: horseshoe -> MSD: 0.04536826995724151 | MSFE: 1.3807551794574473 | ALPL: -30.666279150595443 | elapsed time: 388.530833
Run: 13, M: 3 & prior: horseshoe -> MSD: 0.03870684988536485 | MSFE: 1.5731882658998044 | ALPL: -25.376840828484777 | elapsed time: 388.83286
Run: 11, M: 3 & prior: horseshoe -> MSD: 0.03503329205316674 | MSFE: 1.8001887621943307 | ALPL: -29.911577689650418 | elapsed time: 389.144468
Run: 2, M: 3 & prior: horseshoe -> MSD: 0.05432409543838904 | MSFE: 1.913485099555029 | ALPL: -20.777971811305335 | elapsed time: 389.196067
Run: 3, M

#### M = 3, $p_{0} = 0.20$ and $T = 100$

In [16]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [17]:
multiprocessing_run("svss",M)

Run: 5, M: 3 & prior: svss -> MSD: 0.04254368548612449 | MSFE: 2.6809679790117302 | ALPL: -34.713436866686315 | elapsed time: 139.976903
Run: 4, M: 3 & prior: svss -> MSD: 0.030624552965121832 | MSFE: 0.7080076153575416 | ALPL: -35.36680886061198 | elapsed time: 141.136016
Run: 3, M: 3 & prior: svss -> MSD: 0.03842461730289905 | MSFE: 1.9472200316868042 | ALPL: -35.440143256959736 | elapsed time: 143.877862
Run: 8, M: 3 & prior: svss -> MSD: 0.03489914210870156 | MSFE: 2.955145426688217 | ALPL: -35.09230897964278 | elapsed time: 156.927394
Run: 9, M: 3 & prior: svss -> MSD: 0.037342492609068786 | MSFE: 1.265775854438162 | ALPL: -35.69613198119028 | elapsed time: 163.326898
Run: 14, M: 3 & prior: svss -> MSD: 0.04855269229019527 | MSFE: 1.2998028422759509 | ALPL: -34.69807853585111 | elapsed time: 179.252224
Run: 2, M: 3 & prior: svss -> MSD: 0.060891638182188505 | MSFE: 2.5531182055610886 | ALPL: -34.33004347640682 | elapsed time: 191.658468
Run: 12, M: 3 & prior: svss -> MSD: 0.038410

In [18]:
multiprocessing_run("lasso_alternative",M)

Run: 3, M: 3 & prior: lasso_alternative -> MSD: 0.04033958649072311 | MSFE: 1.886289691034292 | ALPL: -27.221572522916887 | elapsed time: 30.284174
Run: 2, M: 3 & prior: lasso_alternative -> MSD: 0.04899047520189829 | MSFE: 2.1046633201353107 | ALPL: -18.930677190637823 | elapsed time: 32.348476
Run: 11, M: 3 & prior: lasso_alternative -> MSD: 0.029585216333440516 | MSFE: 1.8304520067829921 | ALPL: -27.037458021180868 | elapsed time: 32.562178
Run: 8, M: 3 & prior: lasso_alternative -> MSD: 0.04297870287415935 | MSFE: 2.9874365047949722 | ALPL: -28.369054580090406 | elapsed time: 32.610494
Run: 1, M: 3 & prior: lasso_alternative -> MSD: 0.029908707118085735 | MSFE: 1.4421220230035683 | ALPL: -28.78021968283184 | elapsed time: 32.988277
Run: 4, M: 3 & prior: lasso_alternative -> MSD: 0.039707359800143574 | MSFE: 0.716494024686221 | ALPL: -28.83952547898071 | elapsed time: 33.411833
Run: 5, M: 3 & prior: lasso_alternative -> MSD: 0.029354513232838858 | MSFE: 2.699351452164888 | ALPL: -22

In [19]:
multiprocessing_run("horseshoe",M)

Run: 13, M: 3 & prior: horseshoe -> MSD: 0.03870684988536485 | MSFE: 1.5731882658998044 | ALPL: -25.376840828484777 | elapsed time: 388.173373
Run: 9, M: 3 & prior: horseshoe -> MSD: 0.044164938144558526 | MSFE: 1.2513649390656028 | ALPL: -30.63564209910334 | elapsed time: 388.216418
Run: 14, M: 3 & prior: horseshoe -> MSD: 0.04352989950743131 | MSFE: 1.2845069393355946 | ALPL: -28.79474065830169 | elapsed time: 388.651009
Run: 4, M: 3 & prior: horseshoe -> MSD: 0.04106819019514813 | MSFE: 0.7107971111497506 | ALPL: -29.897230569184096 | elapsed time: 388.938354
Run: 5, M: 3 & prior: horseshoe -> MSD: 0.026060635935937368 | MSFE: 2.6872036002145183 | ALPL: -23.455665257487254 | elapsed time: 388.980118
Run: 12, M: 3 & prior: horseshoe -> MSD: 0.04240937858624262 | MSFE: 1.6021950493502368 | ALPL: -25.209703939282672 | elapsed time: 389.076218
Run: 3, M: 3 & prior: horseshoe -> MSD: 0.03395601301531003 | MSFE: 1.8729640781460317 | ALPL: -28.544904221964767 | elapsed time: 389.132032
Run

#### M = 7, $p_{0} = 0.40$ and $T = 200$

In [20]:
M = 7

In [21]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [22]:
multiprocessing_run("svss",M)

Run: 15, M: 7 & prior: svss -> MSD: 0.02002713930431129 | MSFE: 7.994512075651677 | ALPL: -34.826425947932506 | elapsed time: 850.434291
Run: 12, M: 7 & prior: svss -> MSD: 0.0267255081616881 | MSFE: 5.9432912982088055 | ALPL: -34.369145229553716 | elapsed time: 963.94325
Run: 1, M: 7 & prior: svss -> MSD: 0.025129951171168594 | MSFE: 3.912547880802943 | ALPL: -35.79921285668078 | elapsed time: 1006.754546
Run: 9, M: 7 & prior: svss -> MSD: 0.025997990684097597 | MSFE: 5.953060993015902 | ALPL: -32.63236579049891 | elapsed time: 1053.020211
Run: 6, M: 7 & prior: svss -> MSD: 0.026044271099785656 | MSFE: 4.698970878955717 | ALPL: -35.007485336994414 | elapsed time: 1078.326012
Run: 14, M: 7 & prior: svss -> MSD: 0.019583147019622008 | MSFE: 3.5777552090809293 | ALPL: -35.85890459589311 | elapsed time: 1103.949646
Run: 8, M: 7 & prior: svss -> MSD: 0.022056491310436355 | MSFE: 7.046744755844544 | ALPL: -34.72252082420003 | elapsed time: 1114.271285
Run: 4, M: 7 & prior: svss -> MSD: 0.02

In [23]:
multiprocessing_run("lasso_alternative",M)

Run: 1, M: 7 & prior: lasso_alternative -> MSD: 0.01694145357993569 | MSFE: 3.8316852415505473 | ALPL: -28.741280880971146 | elapsed time: 208.672167
Run: 6, M: 7 & prior: lasso_alternative -> MSD: 0.021496993958305618 | MSFE: 4.594177050218526 | ALPL: -29.718645897999536 | elapsed time: 211.776249
Run: 15, M: 7 & prior: lasso_alternative -> MSD: 0.018823191742188234 | MSFE: 7.597561576351594 | ALPL: -30.77852436059718 | elapsed time: 213.586655
Run: 4, M: 7 & prior: lasso_alternative -> MSD: 0.019185821227369344 | MSFE: 9.03532348387738 | ALPL: -31.339918599772332 | elapsed time: 213.844154
Run: 5, M: 7 & prior: lasso_alternative -> MSD: 0.019934856423996967 | MSFE: 4.88903455138221 | ALPL: -28.888908684727134 | elapsed time: 214.748614
Run: 2, M: 7 & prior: lasso_alternative -> MSD: 0.020998952722245095 | MSFE: 3.980469568936716 | ALPL: -29.515586765837377 | elapsed time: 216.761811
Run: 14, M: 7 & prior: lasso_alternative -> MSD: 0.02027752345158827 | MSFE: 3.5622432704224725 | ALPL

In [24]:
multiprocessing_run("horseshoe",M)

Run: 6, M: 7 & prior: horseshoe -> MSD: 0.021023979029744972 | MSFE: 4.575967408817256 | ALPL: -30.450968415141343 | elapsed time: 2005.86909
Run: 9, M: 7 & prior: horseshoe -> MSD: 0.02246642041568409 | MSFE: 5.370447397372686 | ALPL: -28.596505760034603 | elapsed time: 2025.670383
Run: 5, M: 7 & prior: horseshoe -> MSD: 0.019370340291183792 | MSFE: 4.865286633728935 | ALPL: -29.63817929512538 | elapsed time: 2038.079623
Run: 15, M: 7 & prior: horseshoe -> MSD: 0.0193967548748689 | MSFE: 7.63754023014423 | ALPL: -31.104570441284817 | elapsed time: 2052.000031
Run: 11, M: 7 & prior: horseshoe -> MSD: 0.019222843139646277 | MSFE: 5.398352488348012 | ALPL: -29.97101719989707 | elapsed time: 2055.188229
Run: 8, M: 7 & prior: horseshoe -> MSD: 0.01802474273682977 | MSFE: 6.719080925084286 | ALPL: -30.62626998037044 | elapsed time: 2068.99699
Run: 12, M: 7 & prior: horseshoe -> MSD: 0.022568965933062506 | MSFE: 5.15569740422524 | ALPL: -25.75582934529416 | elapsed time: 2075.194024
Run: 16,

#### M = 7, $p_{0} = 0.20$ and $T = 200$

In [25]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [26]:
multiprocessing_run("svss",M)

Run: 15, M: 7 & prior: svss -> MSD: 0.022343771979766173 | MSFE: 8.028705939101151 | ALPL: -34.221531805558236 | elapsed time: 925.756895
Run: 2, M: 7 & prior: svss -> MSD: 0.015691247721776935 | MSFE: 3.4941242150936547 | ALPL: -35.53611190073549 | elapsed time: 960.779688
Run: 14, M: 7 & prior: svss -> MSD: 0.018537725359463592 | MSFE: 3.1514980357378626 | ALPL: -36.093099209458366 | elapsed time: 1028.415904
Run: 1, M: 7 & prior: svss -> MSD: 0.022273328801429185 | MSFE: 3.5778504125822965 | ALPL: -35.51543327266525 | elapsed time: 1091.359931
Run: 10, M: 7 & prior: svss -> MSD: 0.01955645176321795 | MSFE: 3.333361814269903 | ALPL: -35.42683503454204 | elapsed time: 1100.384156
Run: 8, M: 7 & prior: svss -> MSD: 0.020249281567526204 | MSFE: 4.848169320232046 | ALPL: -34.85979477638423 | elapsed time: 1160.994368
Run: 4, M: 7 & prior: svss -> MSD: 0.01806832867780393 | MSFE: 3.73641115906459 | ALPL: -35.775967704772 | elapsed time: 1266.61614
Run: 16, M: 7 & prior: svss -> MSD: 0.022

In [27]:
multiprocessing_run("lasso_alternative",M)

Run: 1, M: 7 & prior: lasso_alternative -> MSD: 0.02024921733170962 | MSFE: 3.489198951527288 | ALPL: -29.625992926619478 | elapsed time: 193.470944
Run: 10, M: 7 & prior: lasso_alternative -> MSD: 0.021381404019094743 | MSFE: 3.261743641615487 | ALPL: -30.19785647945618 | elapsed time: 210.093485
Run: 16, M: 7 & prior: lasso_alternative -> MSD: 0.020006411568636993 | MSFE: 5.930993554397412 | ALPL: -28.71064934666196 | elapsed time: 212.057098
Run: 15, M: 7 & prior: lasso_alternative -> MSD: 0.022345293706172557 | MSFE: 7.653667866396234 | ALPL: -31.14255775369739 | elapsed time: 212.289396
Run: 5, M: 7 & prior: lasso_alternative -> MSD: 0.02215250061750841 | MSFE: 3.0547183723576232 | ALPL: -27.897896054625626 | elapsed time: 212.995723
Run: 2, M: 7 & prior: lasso_alternative -> MSD: 0.019247879633056936 | MSFE: 3.4717136298208713 | ALPL: -28.414378776108776 | elapsed time: 218.093648
Run: 6, M: 7 & prior: lasso_alternative -> MSD: 0.02358630852194969 | MSFE: 5.354435503045689 | ALPL

In [28]:
multiprocessing_run("horseshoe",M)

Run: 5, M: 7 & prior: horseshoe -> MSD: 0.021532192944053265 | MSFE: 3.0439937374157133 | ALPL: -29.40453764968515 | elapsed time: 1923.390969
Run: 9, M: 7 & prior: horseshoe -> MSD: 0.02223759020451668 | MSFE: 4.34390782884852 | ALPL: -29.185686348221353 | elapsed time: 2010.562743
Run: 6, M: 7 & prior: horseshoe -> MSD: 0.021226480364342843 | MSFE: 5.340325974081729 | ALPL: -30.760626533998668 | elapsed time: 2034.652185
Run: 15, M: 7 & prior: horseshoe -> MSD: 0.02025606991146275 | MSFE: 7.690900420978581 | ALPL: -31.390175361170193 | elapsed time: 2036.688914
Run: 8, M: 7 & prior: horseshoe -> MSD: 0.018224520789354805 | MSFE: 4.803630892753702 | ALPL: -30.282176928207782 | elapsed time: 2054.29817
Run: 7, M: 7 & prior: horseshoe -> MSD: 0.02015937268922945 | MSFE: 2.275043335096208 | ALPL: -29.33629761168679 | elapsed time: 2056.656274
Run: 16, M: 7 & prior: horseshoe -> MSD: 0.01764780637952519 | MSFE: 6.016364174304981 | ALPL: -29.36374916057967 | elapsed time: 2084.878047
Run: 

#### M = 7, $p_{0} = 0.40$ and $T = 100$

In [29]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [30]:
multiprocessing_run("svss",M)

Run: 2, M: 7 & prior: svss -> MSD: 0.022350508994279716 | MSFE: 3.2408972081896987 | ALPL: -35.201602759505136 | elapsed time: 290.42136
Run: 13, M: 7 & prior: svss -> MSD: 0.025649042351692936 | MSFE: 5.014621699190508 | ALPL: -34.70633202034337 | elapsed time: 369.665714
Run: 16, M: 7 & prior: svss -> MSD: 0.022201299554203413 | MSFE: 2.874170676296302 | ALPL: -35.46669984259657 | elapsed time: 384.684494
Run: 7, M: 7 & prior: svss -> MSD: 0.018591898944709636 | MSFE: 2.0582352626229024 | ALPL: -35.495847878462214 | elapsed time: 393.171702
Run: 10, M: 7 & prior: svss -> MSD: 0.027535044208850293 | MSFE: 3.4380783604011578 | ALPL: -35.47452621664077 | elapsed time: 397.44232
Run: 1, M: 7 & prior: svss -> MSD: 0.021924516081104267 | MSFE: 3.1448883418107325 | ALPL: -35.529071600411484 | elapsed time: 461.431662
Run: 8, M: 7 & prior: svss -> MSD: 0.02254597337086895 | MSFE: 2.7753144738365676 | ALPL: -35.04880289319778 | elapsed time: 475.246614
Run: 14, M: 7 & prior: svss -> MSD: 0.01

In [31]:
multiprocessing_run("lasso_alternative",M)

Run: 16, M: 7 & prior: lasso_alternative -> MSD: 0.017920694059802844 | MSFE: 2.839697352510348 | ALPL: -29.004153359996792 | elapsed time: 89.090494
Run: 8, M: 7 & prior: lasso_alternative -> MSD: 0.02579058358081693 | MSFE: 2.8711960605689 | ALPL: -25.99098552895299 | elapsed time: 89.716142
Run: 10, M: 7 & prior: lasso_alternative -> MSD: 0.020195856072779655 | MSFE: 3.2422404606549646 | ALPL: -28.465429785551095 | elapsed time: 92.26752
Run: 1, M: 7 & prior: lasso_alternative -> MSD: 0.016183950483627797 | MSFE: 3.008545125800411 | ALPL: -30.165371893342844 | elapsed time: 93.665996
Run: 13, M: 7 & prior: lasso_alternative -> MSD: 0.02463584824040201 | MSFE: 4.722543057218742 | ALPL: -28.200130478274836 | elapsed time: 94.407005
Run: 5, M: 7 & prior: lasso_alternative -> MSD: 0.021969401039014583 | MSFE: 1.731960730336052 | ALPL: -28.58817128978268 | elapsed time: 94.922385
Run: 15, M: 7 & prior: lasso_alternative -> MSD: 0.02127035949036428 | MSFE: 1.6757223434195863 | ALPL: -29.3

In [32]:
multiprocessing_run("horseshoe",M)

Run: 5, M: 7 & prior: horseshoe -> MSD: 0.02248597352874678 | MSFE: 1.7176292876291972 | ALPL: -29.514767236172126 | elapsed time: 866.846961
Run: 15, M: 7 & prior: horseshoe -> MSD: 0.020988701387081365 | MSFE: 1.6518568251860248 | ALPL: -29.915132498910594 | elapsed time: 878.530255
Run: 14, M: 7 & prior: horseshoe -> MSD: 0.02775469800900334 | MSFE: 2.0619803082475676 | ALPL: -30.32127563533375 | elapsed time: 883.791573
Run: 4, M: 7 & prior: horseshoe -> MSD: 0.024272610195469214 | MSFE: 2.0525162457222352 | ALPL: -30.51044942045144 | elapsed time: 901.79404
Run: 7, M: 7 & prior: horseshoe -> MSD: 0.026036098503380435 | MSFE: 2.055959841207637 | ALPL: -29.120930307605718 | elapsed time: 904.051978
Run: 2, M: 7 & prior: horseshoe -> MSD: 0.017716605962186882 | MSFE: 3.0107329940224923 | ALPL: -29.313084408580295 | elapsed time: 907.085439
Run: 6, M: 7 & prior: horseshoe -> MSD: 0.025106775518443543 | MSFE: 1.8365644129170955 | ALPL: -27.468540940933217 | elapsed time: 911.818735
Run

#### M = 7, $p_{0} = 0.20$ and $T = 100$

In [33]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [34]:
multiprocessing_run("svss",M)

Run: 14, M: 7 & prior: svss -> MSD: 0.01755126130829826 | MSFE: 2.1643185102104407 | ALPL: -35.04384766576243 | elapsed time: 392.790134
Run: 13, M: 7 & prior: svss -> MSD: 0.02092389799592144 | MSFE: 3.8618002683721384 | ALPL: -34.543491072144626 | elapsed time: 411.91551
Run: 2, M: 7 & prior: svss -> MSD: 0.0184197406788605 | MSFE: 1.7465973764507874 | ALPL: -35.77745549235395 | elapsed time: 412.525416
Run: 16, M: 7 & prior: svss -> MSD: 0.017580492870323816 | MSFE: 2.9177089278727375 | ALPL: -35.60193048287197 | elapsed time: 421.773916
Run: 7, M: 7 & prior: svss -> MSD: 0.015232635852046466 | MSFE: 1.8346683903743428 | ALPL: -35.66810545320183 | elapsed time: 426.19371
Run: 11, M: 7 & prior: svss -> MSD: 0.015590652134504161 | MSFE: 1.5896328388610268 | ALPL: -35.507057585800965 | elapsed time: 457.094856
Run: 5, M: 7 & prior: svss -> MSD: 0.019472922533700247 | MSFE: 1.5318737046540098 | ALPL: -35.42589225188735 | elapsed time: 460.545196
Run: 1, M: 7 & prior: svss -> MSD: 0.0166

In [35]:
multiprocessing_run("lasso_alternative",M)

Run: 16, M: 7 & prior: lasso_alternative -> MSD: 0.020005187915600436 | MSFE: 2.8854075131548713 | ALPL: -30.06892317786388 | elapsed time: 91.505157
Run: 1, M: 7 & prior: lasso_alternative -> MSD: 0.017465157374474707 | MSFE: 2.980094331965497 | ALPL: -29.830417038379945 | elapsed time: 92.431634
Run: 8, M: 7 & prior: lasso_alternative -> MSD: 0.026852574690805056 | MSFE: 2.4963865056362566 | ALPL: -25.921835607321235 | elapsed time: 93.116453
Run: 2, M: 7 & prior: lasso_alternative -> MSD: 0.02031299167887625 | MSFE: 1.7377689732986514 | ALPL: -29.447093291723768 | elapsed time: 97.320148
Run: 13, M: 7 & prior: lasso_alternative -> MSD: 0.025514756289814255 | MSFE: 3.7965341874608196 | ALPL: -30.29904295679038 | elapsed time: 97.635478
Run: 9, M: 7 & prior: lasso_alternative -> MSD: 0.032127572128351045 | MSFE: 2.685313638907791 | ALPL: -27.023646805965022 | elapsed time: 98.591964
Run: 10, M: 7 & prior: lasso_alternative -> MSD: 0.020435636204342168 | MSFE: 2.358349458631009 | ALPL:

In [36]:
multiprocessing_run("horseshoe",M)

Run: 10, M: 7 & prior: horseshoe -> MSD: 0.019684430337518877 | MSFE: 2.324217886188737 | ALPL: -30.312621086348525 | elapsed time: 864.137202
Run: 5, M: 7 & prior: horseshoe -> MSD: 0.022329107212481773 | MSFE: 1.5417752742540891 | ALPL: -29.64096717168305 | elapsed time: 865.324622
Run: 15, M: 7 & prior: horseshoe -> MSD: 0.02131358638569903 | MSFE: 1.5928068523434917 | ALPL: -30.36677709219478 | elapsed time: 873.564863
Run: 14, M: 7 & prior: horseshoe -> MSD: 0.024278569737909914 | MSFE: 2.1865004629295797 | ALPL: -30.08655284506104 | elapsed time: 874.282136
Run: 4, M: 7 & prior: horseshoe -> MSD: 0.023968532050088732 | MSFE: 2.2267660411418406 | ALPL: -30.10969014916339 | elapsed time: 890.024217
Run: 2, M: 7 & prior: horseshoe -> MSD: 0.020706508542573755 | MSFE: 1.7218200176541059 | ALPL: -30.398659018083006 | elapsed time: 890.357961
Run: 16, M: 7 & prior: horseshoe -> MSD: 0.01916542636874967 | MSFE: 2.886162413880664 | ALPL: -30.295993866458097 | elapsed time: 893.227338
Run